In [27]:
from sklearn.decomposition import PCA
import numpy as np
from PIL import Image
import os

In [29]:
def dataload(path):
    all_image = []
    re_path = []

    for face_path_name in os.listdir(path):
        #//print(face_path_name)
        if face_path_name == '.DS_Store':                         #guosiqi
            continue
        
        face_path = os.path.join(path, face_path_name)            #./class4-facedata-rgb/guosiqi
        
        for image_path_name in os.listdir(face_path):             #guosiqi.jpg

            image_path = os.path.join(face_path, image_path_name) #./class4-facedata-rgb/guosiqi/guosiqi.jpg
            
            class4_person_dir = os.path.join(path.split('-')[0] + '_recon', face_path_name)
            if not os.path.exists(class4_person_dir):
                os.makedirs(class4_person_dir)
            re_path.append(os.path.join(class4_person_dir, image_path_name))

                
            img_gray = Image.open(image_path).convert('L')
            img_gray.save(os.path.join(os.path.join(path.split('-')[0]+'_recon', face_path_name, image_path_name)))
            img_np = np.array(img_gray)
            all_image.append(img_np)
            
    all_image = np.array(all_image) #(51, 250, 250)
    #//print(all_image.shape)
    all_image_flatten = all_image.reshape((all_image.shape[0], -1)) # (51, 62500) 62500 = 250*250
    #//print(all_image_flatten.shape)
    return all_image_flatten, re_path

all_image_flatten, re_path = dataload('./class4-facedata-rgb')

In [ ]:
len(re_path) # len: 51,  form: './class4_recon/guosiqi/guosiqi.jpg',

In [30]:
def PCA_recon(all_image_flatten, re_path, components_value):
    model = PCA(n_components=components_value) # n_components 不超过 min(n_samples, n_features)
    
    components = model.fit_transform(all_image_flatten)
    
    face_recon = model.inverse_transform(components) # (51, 62500)
    #//print(face_recon.shape)
    # Reshape the reconstructions, gray them and save them.
    for i in range(face_recon.shape[0]):
        each_face_recon = face_recon[i]    
        each_face_recon = Image.fromarray(each_face_recon.reshape((250,250)))
        each_face_recon = each_face_recon.convert('L') #*img_gray
        each_face_recon.save(re_path[i].split('.jpg')[0] + '-recon-' + str(components_value) + '.jpg')
        
        
        
components_value_list = [1, 5, 10, 20, 30, 40, 50]
for components_value in components_value_list:
    PCA_recon(all_image_flatten, re_path, components_value)